In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

%gui asyncio
%config IPCompleter.greedy=True
import apache_beam as beam
from apache_beam.runners.interactive import interactive_runner
from apache_beam.runners.direct import direct_runner
from apache_beam.testing.test_stream import TestStream
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import SetupOptions
from apache_beam.options.pipeline_options import StandardOptions
from apache_beam.options.pipeline_options import DebugOptions
from apache_beam.portability.api import beam_fn_api_pb2
from apache_beam.portability.api import beam_fn_api_pb2_grpc
from apache_beam.portability.api import beam_interactive_api_pb2
from apache_beam.portability.api import beam_interactive_api_pb2_grpc
import time
import cProfile
import grpc

/usr/local/google/home/srohde/Workdir/beam/intbeam/sdks/python/apache_beam/__init__.py:84: UserWarning: Some syntactic constructs of Python 3 are not yet fully supported by Apache Beam.
  'Some syntactic constructs of Python 3 are not yet fully supported by '


In [2]:
from IPython.display import display
play_button = widgets.Button(description="Play")
pause_button = widgets.Button(description="Pause")
step_button = widgets.Button(description="Step")

output = widgets.Output()
display(play_button, output)
display(pause_button, output)
display(step_button, output)

def on_play_button(b):
    with output:
        channel = grpc.insecure_channel('localhost:12345')
        stub = beam_interactive_api_pb2_grpc.InteractiveServiceStub(channel)
        stub.Start(beam_interactive_api_pb2.StartRequest())
        
def on_pause_button(b):
    with output:
        channel = grpc.insecure_channel('loIs an explicit API that the user will know exactly what to expect when they rerun a pipelinecalhost:12345')
        stub = beam_interactive_api_pb2_grpc.InteractiveServiceStub(channel)
        stub.Pause(beam_interactive_api_pb2.PauseRequest())
        
def on_step_button(b):
    with output:
        channel = grpc.insecure_channel('localhost:12345')
        stub = beam_interactive_api_pb2_grpc.InteractiveServiceStub(channel)
        stub.Step(beam_interactive_api_pb2.StepRequest())

play_button.on_click(on_play_button)
pause_button.on_click(on_pause_button)
step_button.on_click(on_step_button)



Button(description='Play', style=ButtonStyle())

Output()

Button(description='Pause', style=ButtonStyle())

Output()

Button(description='Step', style=ButtonStyle())

Output()

In [3]:
from functools import partial
import threading
pipeline_options = PipelineOptions()
pipeline_options.view_as(StandardOptions).streaming = True

def printer(name, x):
    print(name, "produced", x)
    return x

def named_printer(name):
    return partial(printer, name)
    

def run():
    cache_dir = '/usr/local/google/home/srohde/Workdir/beam/intbeam/tmp/cache'
    runner = interactive_runner.InteractiveRunner(
        render_option='mute',
        endpoint='localhost:12345',
        cache_dir=cache_dir)  # render_option could also be 'mute', 'text'
    p = beam.Pipeline(runner=runner, options=pipeline_options)
    pc = p | 'read' >> (TestStream(endpoint='localhost:12345'))
    
    
    
    squares = pc | 'Square' >> beam.Map(lambda x: x*x) | beam.Map(named_printer('square'))
    cubes = pc | 'Cube' >> beam.Map(lambda x: x**3) | beam.Map(named_printer('cubes'))
    result = p.run()
    
t = threading.Thread(target=run)
t.daemon = True
t.start()

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-3-530fa9a6ca9d>", line 27, in run
    result = p.run()
  File "/usr/local/google/home/srohde/Workdir/beam/intbeam/sdks/python/apache_beam/pipeline.py", line 405, in run
    self.to_runner_api(use_fake_coders=True),
  File "/usr/local/google/home/srohde/Workdir/beam/intbeam/sdks/python/apache_beam/pipeline.py", line 659, in to_runner_api
    root_transform_id = context.transforms.get_id(self._root_transform())
  File "/usr/local/google/home/srohde/Workdir/beam/intbeam/sdks/python/apache_beam/runners/pipeline_context.py", line 85, in get_id
    self._id_to_proto[id] = obj.to_runner_api(self._pipeline_context)
  File "/usr/local/google/home/srohde/Workdir/beam/intbeam/sdks/python/apache_beam/pipeline.py", line 880

In [4]:
import os
import tempfile
from apache_beam import coders
from apache_beam.runners.interactive.caching.file_based_cache import TextBasedCache
from google.protobuf import timestamp_pb2
from apache_beam.portability.api.beam_interactive_api_pb2 import InteractiveStreamRecord
from apache_beam.portability.api.beam_runner_api_pb2 import TestStreamPayload
from apache_beam.io.filesystems import FileSystems

def to_timestamp_proto(timestamp_secs):
  """Converts seconds since epoch to a google.protobuf.Timestamp.
  """
  seconds = int(timestamp_secs)
  nanos = int((timestamp_secs - seconds) * 10**9)
  return timestamp_pb2.Timestamp(seconds=seconds, nanos=nanos)


temp_dir = '/usr/local/google/home/srohde/Workdir/beam/intbeam/tmp'
cache_dir = FileSystems.join(temp_dir, 'cache')
print(cache_dir)
file_cache = TextBasedCache(cache_dir)

coder = coders.FastPrimitivesCoder()
elements = []
for i in range(10):
  element = TestStreamPayload.TimestampedElement(
      encoded_element=coder.encode(i), timestamp=i)
  record = InteractiveStreamRecord(
      element=element,
      processing_time=to_timestamp_proto(i),
      watermark=to_timestamp_proto(i))
  elements.append(record.SerializeToString())
file_cache.write(elements)
for r in file_cache.read():
    print(r)


/usr/local/google/home/srohde/Workdir/beam/intbeam/tmp/cache
b'\n\x04\n\x02\x01\x00\x12\x00\x1a\x00'
b'\n\x06\n\x02\x01\x01\x10\x01\x12\x02\x08\x01\x1a\x02\x08\x01'
b'\n\x06\n\x02\x01\x02\x10\x02\x12\x02\x08\x02\x1a\x02\x08\x02'
b'\n\x06\n\x02\x01\x03\x10\x03\x12\x02\x08\x03\x1a\x02\x08\x03'
b'\n\x06\n\x02\x01\x04\x10\x04\x12\x02\x08\x04\x1a\x02\x08\x04'
b'\n\x06\n\x02\x01\x05\x10\x05\x12\x02\x08\x05\x1a\x02\x08\x05'
b'\n\x06\n\x02\x01\x06\x10\x06\x12\x02\x08\x06\x1a\x02\x08\x06'
b'\n\x06\n\x02\x01\x07\x10\x07\x12\x02\x08\x07\x1a\x02\x08\x07'
b'\n\x06\n\x02\x01\x08\x10\x08\x12\x02\x08\x08\x1a\x02\x08\x08'
b'\n\x06\n\x02\x01\t\x10\t\x12\x02\x08\t\x1a\x02\x08\t'
